In [1]:
import os
os.environ['PYTHONPATH']

'./inlaw:research/berry/'

In [2]:
import sys
sys.path

['/Users/tbent/Dropbox/active/confirm/confirmasaurus/anarchy',
 '/Users/tbent/Dropbox/active/confirm/confirmasaurus/anarchy/inlaw',
 '/Users/tbent/Dropbox/active/confirm/confirmasaurus/anarchy/research/berry',
 '/Users/tbent/.mambaforge/envs/imprint/lib/python310.zip',
 '/Users/tbent/.mambaforge/envs/imprint/lib/python3.10',
 '/Users/tbent/.mambaforge/envs/imprint/lib/python3.10/lib-dynload',
 '',
 '/Users/tbent/.mambaforge/envs/imprint/lib/python3.10/site-packages']

In [1]:
import inlaw.nb_util as nb_util
nb_util.setup_nb()

ModuleNotFoundError: No module named 'inlaw'

In [2]:
import jax
import jax.numpy as jnp
import numpy as np
import time
import numpyro
import numpyro.distributions as dist
import numpyro.handlers as handlers
import scipy.stats
import matplotlib.pyplot as plt
from scipy.special import logit, expit

from inlaw import FullLaplace
from inla.berry_model import berry_model, fast_berry


/Users/tbent/.mambaforge/envs/imprint/lib/python3.10/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "
/Users/tbent/.mambaforge/envs/imprint/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
n_arms = 4
fl = inla.FullLaplace(berry_model(n_arms), "sig2", np.zeros((4, 2)))
sig2_rule = util.log_gauss_rule(15, 1e-2, 1e2)
fl = fast_berry(sig2_rule.pts, n_arms)
dtype = np.float64
# for N in 2 ** np.array([4, 9, 14, 16]):
for N in 2 ** np.array([14]):
    y = scipy.stats.binom.rvs(35, 0.3, size=(N, n_arms))
    n = np.full_like(y, 35)
    D = np.stack((y, n), axis=-1).astype(dtype)
    x0 = np.zeros((D.shape[0], sig2_rule.pts.shape[0], 4), dtype=dtype)
    f = lambda: fl(dict(sig2=sig2_rule.pts.astype(dtype), theta=None), D, x0, should_batch=False)
    f()
    for i in range(20):
        start = time.time()
        post, x_max, hess, iters = f()
        end = time.time()
        print(
            f"{N} datasets, {(end - start) / N * 1e6:.3f} us per dataset, {end - start:.2f}s total"
        )

16384 datasets, 2.573 us per dataset, 0.04s total
16384 datasets, 2.487 us per dataset, 0.04s total
16384 datasets, 2.468 us per dataset, 0.04s total
16384 datasets, 2.409 us per dataset, 0.04s total
16384 datasets, 2.364 us per dataset, 0.04s total
16384 datasets, 2.379 us per dataset, 0.04s total
16384 datasets, 2.465 us per dataset, 0.04s total
16384 datasets, 2.383 us per dataset, 0.04s total
16384 datasets, 2.411 us per dataset, 0.04s total
16384 datasets, 2.358 us per dataset, 0.04s total
16384 datasets, 2.393 us per dataset, 0.04s total
16384 datasets, 2.450 us per dataset, 0.04s total
16384 datasets, 2.419 us per dataset, 0.04s total
16384 datasets, 2.517 us per dataset, 0.04s total
16384 datasets, 2.494 us per dataset, 0.04s total
16384 datasets, 2.464 us per dataset, 0.04s total
16384 datasets, 2.367 us per dataset, 0.04s total
16384 datasets, 2.378 us per dataset, 0.04s total
16384 datasets, 2.319 us per dataset, 0.04s total
16384 datasets, 2.508 us per dataset, 0.04s total


In [17]:
cov = np.full((15, n_arms, n_arms), 100.0)
arms = np.arange(n_arms)
cov[:, arms, arms] += sig2_rule.pts[:, None]
neg_precQ = -np.linalg.inv(cov)
np.diagonal(neg_precQ, axis1=1, axis2=2)

array([[-7.09659669e+01, -7.09659669e+01, -7.09659669e+01, -7.09659669e+01],
       [-5.61840290e+01, -5.61840290e+01, -5.61840290e+01, -5.61840290e+01],
       [-3.72804985e+01, -3.72804985e+01, -3.72804985e+01, -3.72804985e+01],
       [-2.10818928e+01, -2.10818928e+01, -2.10818928e+01, -2.10818928e+01],
       [-1.03999608e+01, -1.03999608e+01, -1.03999608e+01, -1.03999608e+01],
       [-4.60704925e+00, -4.60704925e+00, -4.60704925e+00, -4.60704925e+00],
       [-1.89492740e+00, -1.89492740e+00, -1.89492740e+00, -1.89492740e+00],
       [-7.50623441e-01, -7.50623441e-01, -7.50623441e-01, -7.50623441e-01],
       [-2.97564067e-01, -2.97564067e-01, -2.97564067e-01, -2.97564067e-01],
       [-1.22727612e-01, -1.22727612e-01, -1.22727612e-01, -1.22727612e-01],
       [-5.46940527e-02, -5.46940527e-02, -5.46940527e-02, -5.46940527e-02],
       [-2.72664202e-02, -2.72664202e-02, -2.72664202e-02, -2.72664202e-02],
       [-1.56444886e-02, -1.56444886e-02, -1.56444886e-02, -1.56444886e-02],

In [24]:
theta = np.array([-7.21806323, -2.096633  , -0.21382083, -0.06924673])
mu_0 = -1.34
theta_m0 = theta - mu_0
exp_theta_adj = jnp.exp(theta + logit(0.3))
C = 1.0 / (exp_theta_adj + 1)
nCeta = 35 * C * exp_theta_adj
v1 = (np.diagonal(neg_precQ, axis1=1, axis2=2) - nCeta)
v2 = (neg_precQ[:,None, 0,0] - nCeta)
v1, v2

(DeviceArray([[-7.09769617e+01, -7.27168118e+01, -7.99643174e+01, -8.09641537e+01],
              [-5.61950238e+01, -5.79348739e+01, -6.51823795e+01, -6.61822158e+01],
              [-3.72914933e+01, -3.90313434e+01, -4.62788490e+01, -4.72786853e+01],
              [-2.10928876e+01, -2.28327377e+01, -3.00802433e+01, -3.10800796e+01],
              [-1.04109557e+01, -1.21508058e+01, -1.93983114e+01, -2.03981477e+01],
              [-4.61804411e+00, -6.35789419e+00, -1.36053998e+01, -1.46052361e+01],
              [-1.90592226e+00, -3.64577234e+00, -1.08932779e+01, -1.18931142e+01],
              [-7.61618303e-01, -2.50146838e+00, -9.74897398e+00, -1.07488103e+01],
              [-3.08558929e-01, -2.04840901e+00, -9.29591460e+00, -1.02957509e+01],
              [-1.33722474e-01, -1.87357255e+00, -9.12107815e+00, -1.01209144e+01],
              [-6.56889146e-02, -1.80553899e+00, -9.05304459e+00, -1.00528809e+01],
              [-3.82612821e-02, -1.77811136e+00, -9.02561695e+00, -1.0025453

In [ ]:

grad = (
    # dotJI_vmap(neg_precQ_a, neg_precQ_b, theta_m0)
    + jnp.matmul(neg_precQ[None], theta_m0[:, :, :, None])[..., 0]
    + y[:, None]
    - nCeta
)
hess = neg_precQ[None] - ((nCeta * C)[:, :, None, :] * jnp.eye(n_arms))
hess_a = neg_precQ_a[None, :, None] - nCeta * C
hess_b = neg_precQ_b
hess_a = (jnp.diagonal(neg_precQ, axis1=1, axis2=2) - nCeta * C) - hess_b[None, :, None]

![](2022-07-07-20-14-45.png)

In [5]:
import numpy as np
b = 1.0
a = np.random.rand(4)
M = np.full((4,4), b) + np.diag(a)

In [6]:
Minv = np.linalg.inv(M)
Minv

array([[ 1.31148807, -0.06612969, -0.92047326, -0.26805707],
       [-0.06612969,  1.10833818, -0.77032047, -0.22433009],
       [-0.92047326, -0.77032047,  5.47526137, -3.12249854],
       [-0.26805707, -0.22433009, -3.12249854,  3.80766208]])

In [130]:
b = neg_precQ[0,0,1]
a = neg_precQ[0,0,0] - b

In [131]:
@jax.jit
def quad(theta_max, a, b):
    dotprod = ((theta_max.sum(axis=-1) * b)[..., None] + theta_max * a)
    quad = jnp.sum(theta_max * dotprod, axis=-1)
    return quad

In [132]:
%%timeit
quad(theta_max, a, b).block_until_ready()

126 µs ± 1.56 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [129]:
quad(theta_max, a, b)[0,0]

DeviceArray(-0.37294438, dtype=float32)

In [122]:
dotprod2 = jnp.einsum("...ij,...j", neg_precQ, theta_max)
quad2 = jnp.einsum("...i,...ij,...j", theta_max, neg_precQ, theta_max)
quad3 = np.sum(theta_max * dotprod2, axis=-1)

In [123]:
dotprod[0,0], dotprod2[0,0], quad[0,0], quad2[0,0], quad3[0,0]

(DeviceArray([-5.1011925,  1.39013  ,  2.319191 ,  1.390357 ], dtype=float32),
 DeviceArray([-5.101193 ,  1.3901292,  2.319191 ,  1.390357 ], dtype=float32),
 DeviceArray(-0.3729442, dtype=float32),
 DeviceArray(-0.37294447, dtype=float32),
 DeviceArray(-0.37294444, dtype=float32))

In [55]:
inla_obj = inla.INLA(conditional_vmap, grad_hess_vmap, sig2_rule, narms)

In [56]:
theta_max, hess, iters = inla_obj.optimize_loop(data, sig2_rule.pts, 1e-3)
post = inla_obj.posterior(theta_max, hess, sig2_rule.pts, sig2_rule.wts, data)

In [58]:
%%timeit -n 20 -r 5
theta_max, hess, iters = inla_obj.optimize_loop(data, sig2_rule.pts, 1e-3)
post = inla_obj.posterior(theta_max, hess, sig2_rule.pts, sig2_rule.wts, data)

31.4 ms ± 444 µs per loop (mean ± std. dev. of 5 runs, 20 loops each)
